# Eliminating Invalid Inputs/Outliers + Making Dataset

In [ ]:
# Dependencies
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json as json
import typing
import tqdm

In [3]:
import requests
from dotenv import load_dotenv
import os

# load environment variables and functions
load_dotenv()
OAUTH = os.getenv('OAUTH')
CLIENT = os.getenv('CLIENT')

def get_game_from_id(id: str) -> list[str]:
    """
    Get the game name + game id + broadcaster name from the streamer id
    return list of game_id and game_name
    """
    url = "https://api.twitch.tv/helix/channels?broadcaster_id=" + id
    headers = {
        'Authorization': 'Bearer ' + OAUTH,
        'Client-Id': CLIENT
    }
    response = requests.get(url, headers=headers)
    game_id = response.json()['data'][0]['game_id']
    game_name = response.json()['data'][0]['game_name']
    streamer_name = response.json()['data'][0]['broadcaster_name']
    return [game_id, streamer_name, game_name]

In [ ]:
# Let us make a list of all streamer ids by using target.csv
target = pd.read_csv('musae_ENGB_target.csv')
streamer_ids = target['id'].tolist()

# create empty df that connects streamer ID to game ID and game Name
df = pd.DataFrame(columns=['streamer_id', 'streamer_name', 'game_id', 'game_name'])

# loop through all streamer ids and get game id and game name, using tqdm to show progress
for streamer_id in tqdm.tqdm(streamer_ids):
    try:
        streamer_id = str(streamer_id)
        game_id, streamer_name ,game_name = get_game_from_id(streamer_id)
        df = df.append({'streamer_id': streamer_id, 'streamer_name': streamer_name, 'game_id': game_id, 'game_name': game_name}, ignore_index=True)
    except:
        pass

# save df to csv
df.to_csv('streamer_features.csv', index=False)